//+------------------------------------------------------------------+
//|                                             H1_Range_RSI_EA.mq4 |
//|                        Generated by ChatGPT                     |
//+------------------------------------------------------------------+
#property strict

input double RiskPercent = 1.0;       // Risk per trade in %
input int RSI_Period = 14;            // RSI period
input double RSI_Overbought = 70.0;   // RSI overbought level
input double RSI_Oversold = 30.0;     // RSI oversold level
input double Slippage = 3;            // Maximum slippage
input int MagicNumber = 12345;        // EA magic number

double lotSize;

//+------------------------------------------------------------------+
//| Expert initialization function                                   |
//+------------------------------------------------------------------+
int OnInit()
{
   Print("H1 Range RSI EA initialized.");
   return(INIT_SUCCEEDED);
}

//+------------------------------------------------------------------+
//| Expert tick function                                             |
//+------------------------------------------------------------------+
void OnTick()
{
   // Only trigger once per day at 12:01 server time
   datetime currentTime = TimeCurrent();
   int hourNow = TimeHour(currentTime);
   int minuteNow = TimeMinute(currentTime);
   
   static bool orderPlacedToday = false;
   static datetime lastTradeDay = 0;

   if (TimeDay(currentTime) != lastTradeDay)
   {
       orderPlacedToday = false;
       lastTradeDay = TimeDay(currentTime);
   }
   
   if (!orderPlacedToday && hourNow == 12 && minuteNow == 1)
   {
       double H1Open = iOpen(NULL, PERIOD_H1, 1); // Candle 12 PM
       double H1Close = iClose(NULL, PERIOD_H1, 1);
       double H1High = iHigh(NULL, PERIOD_H1, 1);
       double H1Low = iLow(NULL, PERIOD_H1, 1);
       
       double rangeMiddle = (H1High + H1Low) / 2.0;
       double rangeDistance = H1High - rangeMiddle;
       
       // Calculate lot size
       lotSize = CalculateLotSize(rangeDistance);
       
       // Place BuyStop
       OrderSend(Symbol(), OP_BUYSTOP, lotSize, H1High, Slippage, rangeMiddle, 0, "BuyStop H1", MagicNumber, 0, clrBlue);
       
       // Place SellStop
       OrderSend(Symbol(), OP_SELLSTOP, lotSize, H1Low, Slippage, rangeMiddle, 0, "SellStop H1", MagicNumber, 0, clrRed);
       
       orderPlacedToday = true;
       Print("Orders placed for H1 12 PM candle range.");
   }

   ManageTP();
}

//+------------------------------------------------------------------+
//| Lot size calculation based on risk %                              |
//+------------------------------------------------------------------+
double CalculateLotSize(double stopDistance)
{
   double riskMoney = AccountBalance() * RiskPercent / 100.0;
   double lot = riskMoney / (stopDistance * Point * 10); // 10 for USD account, adjust for broker
   lot = NormalizeDouble(lot, 2);
   if (lot < 0.01) lot = 0.01;
   return lot;
}

//+------------------------------------------------------------------+
//| Manage dynamic TP based on RSI                                     |
//+------------------------------------------------------------------+
void ManageTP()
{
   int totalOrders = OrdersTotal();
   for (int i = 0; i < totalOrders; i++)
   {
       if (OrderSelect(i, SELECT_BY_POS, MODE_TRADES) && OrderMagicNumber() == MagicNumber)
       {
           double rsi = iRSI(NULL, PERIOD_H1, RSI_Period, PRICE_CLOSE, 0);
           
           if (OrderType() == OP_BUYSTOP && OrderTakeProfit() == 0)
           {
               if (rsi >= RSI_Overbought)
               {
                   double tpPrice = Bid;
                   OrderModify(OrderTicket(), OrderOpenPrice(), OrderStopLoss(), tpPrice, 0, clrBlue);
               }
           }
           else if (OrderType() == OP_SELLSTOP && OrderTakeProfit() == 0)
           {
               if (rsi <= RSI_Oversold)
               {
                   double tpPrice = Ask;
                   OrderModify(OrderTicket(), OrderOpenPrice(), OrderStopLoss(), tpPrice, 0, clrRed);
               }
           }
       }
   }
}

//+------------------------------------------------------------------+
